In [20]:
import pandas as pd
import pathway as pw

In [21]:
class InputSchema(pw.Schema):
    user_id: int
    date: str
    steps: int
    calories_burned: float
    distance_km: float
    active_minutes: int
    sleep_hours: float
    heart_rate_avg: int
    workout_type: str
    weather_conditions: str
    location: str
    mood: str

input_table = pw.io.csv.read(
    "small_dataset.csv",
    schema=InputSchema,
    mode = "streaming"
)   

In [22]:
user_insights_table = input_table.groupby(pw.this.user_id).reduce(
    user_id=pw.this.user_id,
    total_steps=pw.reducers.sum(pw.this.steps),
    avg_calories_burned=pw.reducers.avg(pw.this.calories_burned),
    total_active_minutes=pw.reducers.sum(pw.this.active_minutes),
    avg_sleep_hours=pw.reducers.avg(pw.this.sleep_hours),
    mood_summary=pw.reducers.max(pw.this.mood),
    workout_count=pw.reducers.count(pw.this.workout_type),
    is_stressed=pw.reducers.any(pw.this.sleep_hours < 6),
    is_unhappy=pw.reducers.any(
        (pw.this.mood == "Stressed") & (pw.this.workout_type == "")
    ),
    heart_attack_alert=pw.reducers.any(
        (pw.this.location == "Home") & (pw.this.heart_rate_avg >= 130)
    ),
    potential_burnout=pw.reducers.any(
        (pw.this.active_minutes > 1000) & (pw.this.sleep_hours < 6)
    ),
    weather_impact=pw.reducers.any(
        (pw.this.weather_conditions == "Rainy") & (pw.this.steps < 5000)
    ),
    risk_of_heart_attack=pw.reducers.any(
        (pw.this.heart_rate_avg > 130) & (pw.this.sleep_hours < 4)
    ),
    overall_fitness_score=pw.reducers.avg(
        pw.this.steps * 0.4 + pw.this.calories_burned * 0.3 + pw.this.active_minutes * 0.2
    ),
)

result_table = user_insights_table.select(
    user_id=pw.this.user_id,
    total_steps=pw.this.total_steps,
    avg_calories_burned=pw.this.avg_calories_burned,
    total_active_minutes=pw.this.total_active_minutes,
    avg_sleep_hours=pw.this.avg_sleep_hours,
    mood_summary=pw.this.mood_summary,
    workout_count=pw.this.workout_count,
    is_stressed=pw.this.is_stressed,
    is_unhappy=pw.this.is_unhappy,
    heart_attack_alert=pw.this.heart_attack_alert,
    potential_burnout=pw.this.potential_burnout,
    weather_impact = pw.this.weather_impact,
    risk_of_heart_attack=pw.this.risk_of_heart_attack,
    overall_fitness_score=pw.this.overall_fitness_score,
)


In [23]:
pw.io.csv.write(result_table, "./tmp/output_stream.csv")
# pw.io.jsonlines.write(result_table, "/tmp/output.jsonlines")
pw.run()

Output()

KeyboardInterrupt: 